In [1]:
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE TypeSynonymInstances, FlexibleInstances #-}

Line 2: Unused LANGUAGE pragma
Found:
{-# LANGUAGE TypeSynonymInstances, FlexibleInstances #-}
Why not:
{-# LANGUAGE FlexibleInstances #-}

#  Interpreters for first-order languages

In usual, we prefer infix expressions like

In [2]:
1919 - (114 + 514)

1291

## Initial embedding

However, when the *initial* embedding encodes expression as a value of algebraic data type:

In [3]:
data Exp = Lit Int
    | Neg Exp
    | Add Exp Exp

The expression above is rewritten with such a prefix notation (sometimes also known as *Polish notation*):

In [4]:
x :: Exp
x = Add (Lit 1919) (Neg (Add (Lit 114) (Lit 514)))

We know the evaluation should be equal to the following prefix expression in Haskell:

In [5]:
(+) 1919 (negate ((+) 114 514))

1291

### Eval

Then, we create the first interpreter, *evaluator*, which interprets expression by case analysis, in other words, *pattern matching*:

In [6]:
eval:: Exp -> Int
eval (Lit n)     = n
eval (Neg e)     = - eval e
eval (Add e1 e2) = eval e1 + eval e2

Let's eval `x` above:

In [7]:
eval x

1291

## Final Embedding

We can embed expressions in a different way:

First, we introduce a representation type for an expression. In the case of `eval`, since the calculation result `Int`, the type used to represent it should also be defined as:

In [8]:
type Repr = Int

Now, use `Repr` as the *target* type for evaluation process, we can directly consider expressions as functions:

In [9]:
lit :: Int -> Repr
lit n = n

neg :: Repr -> Repr
neg e = - e

add :: Repr -> Repr -> Repr
add e1 e2 = e1 + e2

Functions are *compositional*. We can construct an expression represented by those functions and it will be immediately evaluated:

In [10]:
add (lit 1919) (neg (add (lit 114) (lit 514)))

1291

We call this metacircular embedding a *final embedding*, which is dual to *initial embedding*.

### Pretty Print

But when it comes to adding another interpretor, for example, *pretty-printer* to the language, initial embedding seems more *scalable*: we just need to write a new function in which applies new patch-matching process.

In [11]:
view:: Exp -> String
view (Lit n) = show n
view (Neg e) = "(-" ++ view e ++ ")"
view (Add e1 e2) = "(" ++ view e1 ++ " + " ++ view e2 ++ ")"

In [12]:
view x

"(1919 + (-(114 + 514)))"

The `view` interpreter 'evaluates' expressions in a very similar way to `eval`, except that the type of final results is `String` instead of `Int`.

In the final embedding, the evaluator interpreter is hardwired into an expression, which indicates it is impossible to interpret the expression in another way rather that `eval`.

We want the final embedding to support for multiple interpreters as well. One solution offered by Haskell is to write a [*type class*](https://www.haskell.org/tutorial/classes.html) that enables such a parametrization for final result type. 

We define a type class `ExpSYM`, in which every instance `repr` (i.e. final result type of an interpreter) should has the following properties (in this case, can be involved in those 3 functions):

In [13]:
class ExpSYM repr where
    lit :: Int -> repr
    neg :: repr -> repr
    add :: repr -> repr -> repr

In the case of `eval`, the result type is `Int`. So we will specify the how to implement`ExpSYM` instance for `Int` type:

In [14]:
instance ExpSYM Int where
    lit n = n
    neg e = - e
    add e1 e2 = e1 + e2

To map the result type `Int` to actual Haskell `Int` type (alright, it sounds awkward), we define `eval` function as:

In [15]:
eval :: Int -> Int
eval = id

Then we can `eval` the expression above:

In [16]:
eval $ add (lit 1919) (neg (add (lit 114) (lit 514)))

1291

A finally-encoded expression can have multiple ways to interpret. `eval` specifies one of them.

Unlike the initial interpreter:

```hs
eval :: Exp -> Int
```

Our final interpreter uses no pattern-matching. It means no syntax-dispatch overhead, which is similar to [threaded code](https://www.complang.tuwien.ac.at/forth/threaded-code.html) in some way.

`SYM` in `ExpSYM` (name of our type class) stands for *symantics*: the type class defines the syntax about how an expression is embedded (expression form), while an type instance specifies how to interpret the expression. 